In [167]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import *
import pickle
from collections import Counter
from tensorflow.contrib import learn
import data_processor as dp
from nltk import word_tokenize

In [2]:
data = pickle.load(open("/Users/Maximus/downloads/Data_12_03", "rb"))

In [3]:
train_data = data.loc[data.Date < '2009-01-01', :]
dev_data = data.loc[(data.Date >= '2009-01-01') & (data.Date <= '2010-12-31'), :]
test_data = data.loc[data.Date >= '2011-01-01', :]

train_label = train_data.label
dev_label = dev_data.label
test_label = test_data.label

#train_text_features.shape
train_data['Surprise'].fillna(0, inplace=True)
dev_data['Surprise'].fillna(0, inplace=True)
#train_data.fillna(0, inplace=True)

/Users/Maximus/anaconda/envs/tensorflow/lib/python2.7/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [ ]:
import random
random.seed(9)

#randomly drawn negative cases from negative case dataset
randomw_draw = random.sample(range(0, train_data.shape[0]), 8750)
train_data2 = train_data.iloc[randomw_draw,:]

In [208]:
#Generate new labels
label_y = []
for sign in train_data2.label:
  if sign == 'DOWN':
    label_y.append([0, 0, 1])
  elif sign == 'STAY':
    label_y.append([0, 1, 0])
  else:
    label_y.append([0, 1, 0])
label_y = np.asarray(label_y)

In [229]:
reload(data_processor)

<module 'data_processor' from 'data_processor.py'>

In [178]:
x_text = dp.load_features(train_data2.text)

# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

In [200]:
x[5,]

array([  1, 134,  24, 135, 136, 137, 138,  24, 139,  13, 140, 141, 142,
        17, 143, 144,  24, 140, 145,  23,  24,  39, 146, 147,  23,  24,
       148, 149, 155, 156, 157,  17, 158, 159, 146,  24,  25,  28, 227,
       228, 229, 159, 230, 160, 161, 157,  17, 158, 162, 146,  24,  27,
        28, 227, 228, 231, 162, 163, 156, 157,  17, 158, 164, 165, 146,
        24,  27,  28, 227, 228, 231, 164, 165, 230, 163, 156, 157,  17,
       158, 166, 167, 146,  24,  27,  28, 227, 228, 231, 166, 167, 230,
       192, 154, 193, 232,  23, 233,  26,  85, 234, 168, 241, 242, 170,
       298, 299, 300, 246, 298, 236, 173, 174, 175, 176,  24, 112, 179,
       301, 181, 170, 250, 302, 123, 183, 184,  26,  99,  24, 303, 219,
       304, 123, 305, 306, 307,  17, 308, 186, 175, 189, 309,  97,  23,
        24, 183, 184, 237,  42, 310, 278,  24, 311, 141, 188,  40,  81,
       312,  82,  83,  17, 194, 195,  31,  99, 139,  13,  24, 183, 184,
       236, 189, 183,   0,   0,   0,   0,   0,   0,   0,   0,   

In [225]:
# Create train and development set

x_train, x_dev = x[:7000], x[7000:]
y_train, y_dev = label_y[:7000], label_y[7000:]

In [235]:
import datetime
from cnn_net import CNN
# Training
# ==================================================

##
# Hyperparameters
V = 10000
M = 50  #Embedding Size
FS = [3,4,5] #Filter Sizes
Num_F = 3
dropout_keep_prob = 1
L = 0
batch_size = 64
num_epoch = 100


with tf.Graph().as_default():
   
    sess = tf.Session(config=tf.ConfigProto(device_filters="/cpu:0"))
    with sess.as_default():
        cnn = CNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size = M,
            filter_sizes = FS,
            num_filters = Num_F,
            l2_reg_lambda = L)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)


        # Output directory for models and summaries
        #timestamp = str(int(time.time()))
        #out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        #print("Writing to {}\n".format(out_dir))

        
        # Summaries for loss and accuracy
        #loss_summary = tf.scalar_summary("loss", cnn.loss)
        #acc_summary = tf.scalar_summary("accuracy", cnn.accuracy)

        # Train Summaries
        #train_summary_op = tf.merge_summary([loss_summary, acc_summary, grad_summaries_merged])
        #train_summary_dir = os.path.join(out_dir, "summaries", "train")
        #train_summary_writer = tf.train.SummaryWriter(train_summary_dir, sess.graph)


        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        #checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        #checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        #if not os.path.exists(checkpoint_dir):
        #    os.makedirs(checkpoint_dir)
        #saver = tf.train.Saver(tf.all_variables())

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.initialize_all_variables())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step,loss, accuracy = sess.run(
                [train_op, global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            #time_str = datetime.datetime.now().isoformat()
            #if step % 200 == 0:
            #  print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

        
        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, loss, accuracy = sess.run(
                [global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))        
           
        # Generate batches
        batches = data_processor.batch_iter(
            list(zip(x_train, y_train)), batch_size, num_epoch)
        # Training loop. For each batch...
        
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            if current_step % 500 == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev)



Evaluation:
2016-12-05T23:15:37.881951: step 500, loss 0.639436, acc 0.683429

Evaluation:
2016-12-05T23:16:23.612324: step 1000, loss 0.658692, acc 0.656

Evaluation:
2016-12-05T23:17:10.110645: step 1500, loss 0.724114, acc 0.615429

Evaluation:
2016-12-05T23:18:19.495277: step 2000, loss 0.819129, acc 0.641714

Evaluation:
2016-12-05T23:19:33.684980: step 2500, loss 0.887047, acc 0.608571

Evaluation:
2016-12-05T23:20:42.530167: step 3000, loss 0.970568, acc 0.606286

Evaluation:
2016-12-05T23:21:50.556228: step 3500, loss 1.07028, acc 0.617714

Evaluation:
2016-12-05T23:22:57.234588: step 4000, loss 1.18787, acc 0.62

Evaluation:
2016-12-05T23:23:51.937879: step 4500, loss 1.25628, acc 0.605143

Evaluation:
2016-12-05T23:24:52.248761: step 5000, loss 1.33099, acc 0.613143

Evaluation:
2016-12-05T23:25:56.024238: step 5500, loss 1.43244, acc 0.615429

Evaluation:
2016-12-05T23:26:54.973597: step 6000, loss 1.51789, acc 0.614857

Evaluation:
2016-12-05T23:27:51.295032: step 6500, lo

In [127]:
#print sum(max_document_length)/float(len(max_document_length))
#print max_document_length
print sum(max_document_length)/float(len(max_document_length))

84.4157714286


In [109]:
test = dp.load_features(train_data2.text)

In [112]:
del test[:]

In [95]:
train_data.text.shape[0]

17500

In [93]:
train_data.text.iloc[0]

'\n\nTEXT:\n8\nFORM\n-K\nCURRENT\nREPORT\n13 15\nPURSUANT\nTO\nSECTION\nOR\n(\nd\n)\nOF\nTHE\n1934\nSECURITIES\nEXCHANGE\nACT\nOF\nCheck the appropriate box below if the Form 8-K filing is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions:\n|_| Written communications pursuant to Rule 425 under the Securities Act (17 CFR 230.425)\n|_| Soliciting material pursuant to Rule 14a-12 under the Exchange Act (17 CFR 240.14a-12)\n|_| Pre-commencement communications pursuant to Rule 14d-2(b) under the Exchange Act      (17 CFR 240.14d-2(b))\n|_| Pre-commencement communications pursuant to Rule 13e-4(c) under the Exchange Act      (17 CFR 240.13e-4(c))\n  \nOn December 7, 2004, Comtech Telecommunications Corp. (the "Registrant") issued a press release announcing its results of operations for its first quarter ended October 31, 2004. A copy of the press release issued by the Registrant concerning the foregoing results is furnished herew

In [ ]:
x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

In [85]:
test[]

"The information contained in this report (including the exhibit hereto) shall not be deemed filed under the Securities and Exchange Commission's rules and regulations and shall not be deemed incorporated by reference in any filing under the Securities Act of 1933, as amended."

In [65]:
train_data.bow[13]

'Counter({\'-\': 22, \'million\': 13, \'31,\': 10, \'telecommunications\': 10, \'communications\': 9, \'october\': 9, \'three\': 8, \'ended\': 8, \'comtech\': 8, \'months\': 8, \'results\': 7, \'corp.\': 7, \'exchange\': 7, \'quarter\': 7, \'securities\': 7, \'act\': 7, \'fiscal\': 7, \'microwave\': 6, \'impact\': 6, \'operating\': 6, \'systems\': 6, \'first\': 6, \'pursuant\': 6, \'conference\': 6, \'2004\': 6, \'net\': 5, \'data\': 5, \'mobile\': 5, \'inability\': 5, \'cfr\': 4, \'strong\': 4, \'call\': 4, \'company\': 4, \'over-the-horizon\': 4, \'earnings\': 4, \'products,\': 4, \'|_|\': 4, "company\'s": 4, \'exhibit\': 4, \'performance\': 4, \'subject\': 4, \'cash\': 4, \'(17\': 4, \'sales\': 4, \'rule\': 4, \'2005\': 4, \'issued\': 3, \'activities\': 3, \'2004.\': 3, \'december\': 3, \'information\': 3, \'contracts\': 3, \'chief\': 3, \'operations\': 3, \'including\': 3, \'increase\': 3, \'7,\': 3, \'filing\': 3, \'report\': 3, \'release\': 3, \'recently\': 3, \'due\': 3, \'balan

In [68]:
#from collections import Counter
test = eval(train_data.bow[13])

In [7]:
import random
random.seed(9)

#randomly drawn negative cases from negative case dataset
randomw_draw = random.sample(range(0, train_data.shape[0]), 8750)
train_data2 = train_data.iloc[randomw_draw,:]

In [66]:
cv = CountVectorizer(vocabulary=['hot', 'cold', 'old'])
cv.fit_transform(['pease porridge cold', 'pease porridge hot', 'pease porridge in the pot old', 'nine days old']).toarray()


array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1]])

In [48]:
vectorizer = CountVectorizer(min_df=1, stop_words='english')
train_text_features = vectorizer.fit_transform(train_data2.bow)

In [60]:
vectorizer.vocabulary_

{u'reallocations': 56428,
 u'localizer': 44406,
 u'degussa': 28681,
 u'lumire': 44821,
 u'rudnay': 59177,
 u'measurespresented': 46185,
 u'woods': 70925,
 u'hanging': 37499,
 u'2003revenues': 4032,
 u'localized': 44405,
 u'5989': 10722,
 u'caney': 23252,
 u'5980': 10716,
 u'256909': 5403,
 u'5986': 10720,
 u'5984': 10718,
 u'refunding': 57079,
 u'crossbar': 27486,
 u'103541': 1414,
 u'103540': 1413,
 u'severstal': 60694,
 u'11878005': 2028,
 u'broward': 22443,
 u'bringing': 22307,
 u'cryolife': 27561,
 u'238trading': 5139,
 u'wooden': 70912,
 u'guertin': 37147,
 u'4711006': 8875,
 u'wednesday': 70193,
 u'circuitry': 24880,
 u'operationsincome': 50844,
 u'shows': 61141,
 u'immunities': 39527,
 u'networkwhich': 48878,
 u'0053': 107,
 u'fiture': 34505,
 u'0057': 111,
 u'0056': 110,
 u'0055': 109,
 u'0054': 108,
 u'270': 5631,
 u'271': 5646,
 u'272': 5675,
 u'273': 5687,
 u'274': 5700,
 u'275': 5717,
 u'276': 5732,
 u'277': 5745,
 u'278': 5758,
 u'279': 5769,
 u'edical': 31300,
 u'errors':

In [63]:
train_text_features.toarray()[:1]

array([[0, 1, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]])

In [62]:
print a[:900]

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
def sentence_clean(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def data_processing(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
return [x_text, y]

In [56]:
train_data['label2'] = train_data['label'].apply(lambda x: 1 if str(x) == 'UP' else 0)

/Users/Maximus/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [57]:
train_data['label2'].shape

(17500,)

In [4]:
vectorizer = CountVectorizer(min_df=1, stop_words='english')
train_text_features = vectorizer.fit_transform(train_data.bow)

dev_text_features = vectorizer.transform(dev_data.bow)

In [5]:
all_data_train = np.append(train_text_features.toarray(),\
                           np.array(train_data.Surprise).reshape(len(train_data.Surprise),1), 1)
all_data_dev = np.append(dev_text_features.toarray(),\
                         np.array(dev_data.Surprise).reshape(len(dev_data.Surprise),1), 1)

In [43]:
tf.reset_default_graph()  # clear away any leftovers from previous executions

input_dim = all_data_train.shape[1]
#train_label

X_ = tf.placeholder(tf.float32, shape=[None, input_dim], name="X")
y_ = tf.placeholder(tf.float32, shape=[None], name="y")

# Variables for the parameters of our model
with tf.name_scope('model_parameters'):
    w_ = tf.Variable(tf.zeros([input_dim, 1], dtype=tf.float32), name="w")
    b_ = tf.Variable(0.0, dtype=tf.float32, name="b")

# Output layer: \sigma(Xw + b)
with tf.name_scope('logit'):
    logits_ = tf.squeeze(tf.matmul(X_, w_)) + b_
with tf.name_scope('pred_proba'):
    pred_proba_ = tf.sigmoid(logits_)

# Cost function
with tf.name_scope('cost_function'):
    loss_ = tf.nn.sigmoid_cross_entropy_with_logits(logits_, y_, name='loss')

In [44]:
# Gradient descent training
with tf.name_scope('training'):
    alpha_ = tf.placeholder(tf.float32, name="learning_rate")
    optimizer_ = tf.train.GradientDescentOptimizer(alpha_)
    train_step_ = optimizer_.minimize(loss_)

# Initializer step
init_ = tf.initialize_all_variables()

In [ ]:
max_steps = 100
print_every = max_steps / 10
alpha = 0.01  # learning rate

# We'll constrain this to run on CPU, as GPUs aren't very efficient
# on a model this small.
session = tf.Session(config=tf.ConfigProto(device_filters="/cpu:0"))
session.run(init_)  # initialize variables for this session


for i in xrange(max_steps):          
    # Run a single gradient descent step
    c, p, _ = session.run([loss_, pred_proba_, train_step_],
                           feed_dict={X_: all_data_train, y_: train_data.label2, alpha_: alpha})
        
    if (i % print_every == 0):
        avg_cost = sum(c) / len(train_data.label2)
        print "[iter %d] Average cost: %.03f" % (i, avg_cost)
        

# Run model over training set one last time
costs, y_pred = session.run([loss_, pred_proba_], 
                            feed_dict={X_: all_data_train, y_: train_data.label2})

print ""
print "Accuracy: %.03f" % np.mean(train_data.label2 == (y_pred >= 0.5))
print "Mean cross-entropy loss: %.03f" % np.mean(costs)


[iter 0] Average cost: 0.693


In [ ]:
fig = plt.figure()
plot_model(X, y, lambda X: session.run(pred_proba_, feed_dict={X_:X}))
plt.title("%d steps (final)" % max_steps)